In [1]:
# | exporti
from langchain_openai import ChatOpenAI

from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from langchain.chains import ConversationChain

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ChatMessageHistory

from IPython.display import clear_output
from langchain_core.output_parsers import StrOutputParser

from langchain.memory import ConversationBufferWindowMemory
from langchain.callbacks.base import BaseCallbackHandler

from typing import Any
from queue import Queue, Empty
from threading import Thread


# def handle_streaming_ref(stream):
#     response = []
#     for chunk in stream:
#         print(chunk, end="|", flush=True)
#         response.append(chunk.content)

#         clear_output(wait=True)
#         response = "".join(response).strip()

#         return response

# q = Queue()
job_done = object()


class QueueCallback(BaseCallbackHandler):
    """Callback handler for streaming LLM responses to a queue."""

    def __init__(self, q):
        self.q = q

    def on_llm_new_token(self, token: str, **kwargs: Any) -> None:
        self.q.put(token)

    def on_llm_end(self, *args, **kwargs: Any) -> None:
        return self.q.empty()


callbacks = [QueueCallback(q)]

llm = ChatOpenAI(
    openai_api_base="http://172.20.2.239:1234/v1/",
    openai_api_key="not-needed",
    model="local_model",
    callbacks=callbacks,
    temperature=0.7,
)


def generate_promptchain_for_string(system_prompt, llm) -> ConversationChain:
    parser = StrOutputParser()
    prompt = ChatPromptTemplate.from_messages(
        [
            SystemMessage(content=system_prompt),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )

    return prompt | llm | parser


def get_llm_response(
    user_content,
    llm,
    q,
    chat_history=None,
    system_prompt="You are a QandA Bot",
) -> AIMessage:
    chat_history = chat_history or ChatMessageHistory()
    chat_history.add_user_message(user_content)

    chain = generate_promptchain_for_string(system_prompt=system_prompt, llm=llm)

    def task():
        response = chain.stream({"messages": chat_history.messages})
        q.put(job_done)

    t = Thread(target=task)
    t.start()


def main():
    q = Queue()
    get_llm_response(
        user_content="translate i love programming into french", llm=llm, q=q
    )
    s = ""
    print("here")
    while True:
        try:
            next_token = q.get(True, timeout=1)
            if next_token is job_done:
                print("job_done")
                break
            s += next_token
            print(s)
            yield s

        except Empty:
            continue

        except Exception as e:
            print(e)

    # print(q)


main()

NameError: name 'q' is not defined